In [1]:
import json
import openpyxl
import requests
from tqdm import tqdm
import time
from bs4 import BeautifulSoup

In [2]:

# / +str은 series_id를 사이트에 연결 해야하는데 series_id는 사실상 정수라 문자열로 반환 후 연결해줘야함
epi_url = 'https://webtoon.kakao.com'

# epi_url로 저장한 url에 HTTP GET 요청을 하는것
epi_res = requests.get(epi_url)

# .content인 이유는 epi_res는 실제 콘텐츠가 아닌 응답한 객체/ 실제 content를 가져오기 위해 사용 
soup = BeautifulSoup(epi_res.content, 'html.parser')

# 카카오웹툰서 사용하는 고유 식별자 => CSS
result = soup.select_one('#__NEXT_DATA__').get_text()

# 데이터 로드
json_data = json.loads(result)

In [ ]:
json_data


In [3]:
import json
import openpyxl
import requests
from tqdm import tqdm
import time
from bs4 import BeautifulSoup

#           
def epi_info(series_id, state):
    # 'series_id'는 카카오 플랫폼의 각 웹툰 시리즈에 대한 고유 식별자 => 각 웹툰에 부여된 숫자(4자리의 정수)
    #  웹크롤링은 불법스캔으로 막아놔서 text로 출력
    # content/text는 웹툰 에피소드 콘텐츠의 텍스트에 접근하는 방법
    
    # / +str은 series_id를 사이트에 연결 해야하는데 series_id는 사실상 정수라 문자열로 반환 후 연결해줘야함
    epi_url = 'https://webtoon.kakao.com/content/text/' + str(series_id)
    
    # epi_url로 저장한 url에 HTTP GET 요청을 하는것
    epi_res = requests.get(epi_url)

    # .content인 이유는 epi_res는 실제 콘텐츠가 아닌 응답한 객체/ 실제 content를 가져오기 위해 사용 
    soup = BeautifulSoup(epi_res.content, 'html.parser')
    
    # 카카오웹툰서 사용하는 고유 식별자 => CSS
    result = soup.select_one('#__NEXT_DATA__').get_text()
    
    # 데이터 로드
    json_data = json.loads(result)

    
    series_title = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['title']
    genre = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['genre']
    authors = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['authors']
    viewCount = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['viewCount']
    likeCount = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['likeCount']
    isAdult = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['isAdult']
    synopsis = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['synopsis']
    try:
        publisher = [value['names'][0] for value in json_data['props']['initialState']['content']['contentMap'][str(series_id)]['authorDetails'] if value['type']=='publisher'][0]
    except:
        publisher = '-'
        
    return [
        series_id,
        series_title,
        authors,
        publisher,
        isAdult,
        viewCount,
        likeCount,
        state,
        genre,
        synopsis
    ]


In [ ]:

# 완결작 수집
def get_completed():
    state = '완결'
    url = 'https://gateway-kw.kakao.com/section/v1/timetables/days?placement=timetable_completed'
    res = requests.get(url)
    contents = res.json()['data'][0]['cardGroups'][0]['cards']

    wb = openpyxl.Workbook()
    ws = wb.active
    ws.append([
        'series_id',
        'series_title',
        'authors',
        'publisher',
        'isAdult',
        'viewCount',
        'likeCount',
        'state',
        'genre',
        'synopsis'
    ])

    error_page = []

    for series in tqdm(contents):
        series_id = series['content']['id']

        try:
            ws.append(epi_info(series_id, state))
            time.sleep(3)

        except:
            print(series_id)
            error_page.append(series_id)


    wb.save('카카오웹툰_완결_{}.xlsx'.format(2))
    print(error_page)

get_completed()


In [ ]:

    
import json
import openpyxl
import requests
from tqdm import tqdm
import time
from bs4 import BeautifulSoup

# def epi_info(series_id, state):
#     # 'series_id'는 카카오 플랫폼의 각 웹툰 시리즈에 대한 고유 식별자 => 각 웹툰에 부여된 숫자(4자리의 정수)
#     #  웹크롤링은 불법스캔으로 막아놔서 text로 출력
#     # content/text는 웹툰 에피소드 콘텐츠의 텍스트에 접근하는 방법
    
#     # / +str은 series_id를 사이트에 연결 해야하는데 series_id는 사실상 정수라 문자열로 반환 후 연결해줘야함
#     epi_url = 'https://webtoon.kakao.com/content/text/' + str(series_id)
    
#     # epi_url로 저장한 url에 HTTP GET 요청을 하는것
#     epi_res = requests.get(epi_url)

#     # .content인 이유는 epi_res는 실제 콘텐츠가 아닌 응답한 객체/ 실제 content를 가져오기 위해 사용 
#     soup = BeautifulSoup(epi_res.content, 'html.parser')
    
#     # 카카오웹툰서 사용하는 고유 식별자 => CSS
#     result = soup.select_one('#__NEXT_DATA__').get_text()
    
#     # 데이터 로드
#     json_data = json.loads(result)
#     series_title = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['title']
#     genre = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['genre']
#     authors = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['authors']
#     viewCount = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['viewCount']
#     likeCount = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['likeCount']
#     isAdult = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['isAdult']
#     synopsis = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['synopsis']
#     try:
#         publisher = [value['names'][0] for value in json_data['props']['initialState']['content']['contentMap'][str(series_id)]['authorDetails'] if value['type']=='publisher'][0]
#     except:
#         publisher = '-'
        
#     return [
#         series_id,
#         series_title,
#         authors,
#         publisher,
#         isAdult,
#         viewCount,
#         likeCount,
#         state,
#         genre,
#         synopsis
#     ]

# # 완결작 수집
# def get_completed():
#     state = '완결'
#     url = 'https://gateway-kw.kakao.com/section/v1/timetables/days?placement=timetable_completed'
#     res = requests.get(url)
#     contents = res.json()['data'][0]['cardGroups'][0]['cards']

#     wb = openpyxl.Workbook()
#     ws = wb.active
#     ws.append([
#         'series_id',
#         'series_title',
#         'authors',
#         'publisher',
#         'isAdult',
#         'viewCount',
#         'likeCount',
#         'state',
#         'genre',
#         'synopsis'
#     ])

#     error_page = []

#     for series in tqdm(contents):
#         series_id = series['content']['id']

#         try:
#             ws.append(epi_info(series_id, state))
#             time.sleep(3)

#         except:
#             print(series_id)
#             error_page.append(series_id)


#     wb.save('카카오웹툰_완결_{}.xlsx'.format(2))
#     print(error_page)

# get_completed()


In [5]:
def get_series():
    # 연재작 수집
    state = ['연재_월요일']
    url = 'https://gateway-kw.kakao.com/section/v1/timetables/days?placement=timetable_mon'

    res = requests.get(url)
    contents = res.json()['data'][0]['cardGroups'][0]['cards']

    wb = openpyxl.Workbook()
    ws = wb.active
    ws.append([
        'series_id',
        'series_title',
        'authors',
        'publisher',
        'isAdult',
        'viewCount',
        'likeCount',
        'state',
        'genre',
        'synopsis'
    ])

    error_page = []

    for series in tqdm(contents):
        series_id = series['content']['id']

        try:
            ws.append(epi_info(series_id, state))
            time.sleep(3)

        except:
            print(series_id)
            error_page.append(series_id)


    wb.save('카카오웹툰_{}.xlsx'.format(state))
    print(error_page)

get_series()

  1%|          | 1/124 [00:00<00:58,  2.11it/s]

2489


  2%|▏         | 2/124 [00:01<01:08,  1.77it/s]

818


  2%|▏         | 3/124 [00:01<00:59,  2.04it/s]

...

2929


 99%|█████████▉| 123/124 [01:30<00:00,  2.26it/s]

3313


100%|██████████| 124/124 [01:30<00:00,  1.36it/s]

3410
[2489, 818, 1781, 2473, 2341, 2043, 1959, 3425, 3423, 2901, 3406, 3405, 3331, 3395, 3396, 2983, 2121, 1985, 3355, 2385, 2748, 198, 2745, 2715, 2862, 2769, 2467, 2981, 2589, 3360, 2477, 2469, 1931, 3384, 3138, 2147, 3060, 3237, 2329, 2143, 3352, 2785, 2658, 2381, 1703, 2916, 2797, 2760, 2673, 2302, 1716, 2683, 1621, 1078, 2430, 3309, 2772, 2524, 3219, 3294, 3045, 2553, 1140, 3345, 1768, 2598, 3346, 2666, 1392, 3212, 2727, 2592, 3004, 2528, 3010, 3351, 2434, 3067, 3319, 3253, 3170, 3186, 3276, 2888, 3084, 2818, 2805, 3196, 2967, 3218, 3146, 3070, 3327, 3236, 3169, 2694, 2870, 2851, 2827, 3278, 3027, 3194, 3329, 2649, 3250, 2630, 3293, 3275, 2509, 3118, 2803, 2552, 2763, 3031, 3328, 3168, 3119, 1847, 2583, 2622, 2906, 2929, 3313, 3410]
